**Mini-batching**:
Uno dei più grandi problemi del processare i dati singolarmente, cioè uno per volta è che non possiamo scalare bene in ambito big data. Processando un solo elemento alla volta non possiamo sfruttare i vantaggi della vettorizzazione, cioè tutti quei metodi che abbiamo imparato nella versione batch.

Quindi vedendo tutto dall'alto noi `River` utilizza un ciclo `for loop`, che diciamo può soffrire in taluni casi quando sono richieste velocità maggiori... Questo non per dire che `river` è lento, diciamo che invece per processare un elemento alla volta è molto più veloce di altre librerie... quindi possiamo dire che river è veloce in questo contesto anche perchè le altre librerie si basano più su dati batches.

Nel prossimo esempio vedremo e costruiremo una Pipeline che ci permette di scalare e trainare un regressore logistico


In [1]:
from river import compose
from river import linear_model
from river import preprocessing

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression()
)

In [2]:
from river import datasets

dataset = datasets.Higgs()
if not dataset.is_downloaded:
    dataset.download()
dataset

Higgs dataset.

The data has been produced using Monte Carlo simulations. The first 21 features (columns 2-22)
are kinematic properties measured by the particle detectors in the accelerator. The last seven
features are functions of the first 21 features; these are high-level features derived by
physicists to help discriminate between the two classes.

      Name  Higgs                                                                       
      Task  Binary classification                                                       
   Samples  11,000,000                                                                  
  Features  28                                                                          
    Sparse  False                                                                       
      Path  /home/giodefa/river_data/Higgs/HIGGS.csv.gz                                 
       URL  https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
      Size  2.62 GB    

In [4]:
import pandas as pd

names = [
    'target', 'lepton pT', 'lepton eta', 'lepton phi',
    'missing energy magnitude', 'missing energy phi',
    'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag',
    'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag',
    'jet 3 pt', 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag',
    'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag',
    'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb'
]


In [5]:
for x in pd.read_csv(dataset.path, names=names, chunksize=8096, nrows=3e5):
    y = x.pop('target')
    y_pred = model.predict_proba_many(x)
    model.learn_many(x, y)

In [6]:
import importlib
import inspect

def can_mini_batch(obj):
    return hasattr(obj, 'learn_many')

for module in importlib.import_module('river').__all__:
    if module in ['datasets', 'synth']:
        continue
    for name, obj in inspect.getmembers(importlib.import_module(f'river.{module}'), can_mini_batch):
        print(name)

OneClassSVM
MiniBatchClassifier
MiniBatchRegressor
MiniBatchTransformer
SKL2RiverClassifier
SKL2RiverRegressor
Pipeline
Select
TransformerProduct
TransformerUnion
BagOfWords
TFIDF
LinearRegression
LogisticRegression
Perceptron
OneVsRestClassifier
BernoulliNB
ComplementNB
MultinomialNB
MLPRegressor
OneHotEncoder
StandardScaler
